In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

## Import data

1 - epilepsy
0 - control

In [24]:
eeg_points: list[str] = ['AF3', 'AF4', 'F3', 'F4', 'F7', 'F8', 'FC5', 'FC6', 'O1', 'O2', 'P7', 'P8', 'T7', 'T8']
label_encoder: dict[str, int] = {
    'epilepsy' : 1, 
    "control" : 0
}

studies: list[pd.Series] = []

In [3]:
# Guinea Bissau
meta_gb = pd.read_csv("DATASET/metadata_guineabissau.csv")

In [12]:
def get_closed_eyes_signal(signal: pd.DataFrame) -> pd.DataFrame:
    num_rows: int = len(signal)
    return signal[int(3 * num_rows / 5):]

In [25]:
for row in meta_gb.iterrows():
    subject_id = row[1]['subject.id']
    label = label_encoder[row[1].Group.lower()]
    signal = pd.read_csv(f"DATASET/EEGs_Guinea-Bissau/signal-{subject_id}.csv")
    signal = signal[eeg_points]
    
    closed_eyes_signal = get_closed_eyes_signal(signal)

    studies.append(closed_eyes_signal)

In [39]:
target = studies[0]
target_df = pd.DataFrame(target)

correlation_matrix = target_df.corr()

In [42]:
# Nigeria
meta_nig = pd.read_csv("DATASET/metadata_nigeria.csv")

In [43]:
for _, row in meta_nig.iterrows():
    subject_id = row['subject.id']
    label = label_encoder[row.Group]
    filename = row['csv.file'][:-3]
    signal = pd.read_csv(f"DATASET/EEGs_Nigeria/{filename}")
    signal = signal[eeg_points]
    
    closed_eyes_signal = get_closed_eyes_signal(signal)

    studies.append(closed_eyes_signal)

In [45]:
len(studies)

318

In [46]:
print(type(studies[0]))

<class 'pandas.core.frame.DataFrame'>


In [50]:
for study in studies:
    print(type(study))

IndentationError: expected an indented block after function definition on line 1 (1399594715.py, line 2)

## Train data

In [8]:
from models import GATv2Lightning
from train import kfold_cval

In [9]:
import torch

In [ ]:
print(torch.cuda.is_available())

In [11]:
class Args:
    timestep = 6
    epochs = 25
    batch_size = 256
    n_splits = 5
    cache_dir = "data/cache"
    exp_name = "eeg_exp"

args = Args()

In [ ]:
kfold_cval(signal_shapes, args)